In [75]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import glob
import pylab as pl
from sklearn.cluster import KMeans
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import tree 
from sklearn import preprocessing
import plotly.express as px
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix,roc_auc_score,roc_curve
import matplotlib.pyplot as plt
import nltk
from sklearn.model_selection import GridSearchCV
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to C:\Users\Zhang
[nltk_data]     Lu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Zhang
[nltk_data]     Lu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Part 1: Text cleaning

In [174]:
News_DF=pd.read_csv('News_Headline.csv', error_bad_lines=False, encoding='latin-1')

D:\anaconda3\envs\tensorflow1\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [175]:
News_DF

,category,headline,links,short_description,keywords,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,POLITICS,GOP Congressman Suggests Looser Standard Of Pr...,https://www.huffingtonpost.com/entry/zeldin-mu...,"Rep. Lee Zeldin says there needs to be ""some t...",zeldin-muslims-reasonable-suspicion,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POLITICS,The ACLU Is Suing To Strike Down Ohio's Congre...,https://www.huffingtonpost.com/entry/ohio-cong...,The civil liberties group says Ohio's map is s...,ohio-congressional-map-gerrymandering,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POLITICS,Don't Like The New Climate Rules? Thank Congress.,https://www.huffingtonpost.com/entry/congress-...,Legislators could have passed a climate change...,congress-climate-rules-emissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POLITICS,The Story Behind An Iconic Picture Of Civil Ri...,https://www.huffingtonpost.com/entry/how-julia...,An often-forgotten battle for free speech righ...,how-julian-bond-fought-georgia-all-the-way-to-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POLITICS,Pro-Abortion Rights Progressive Wins Nebraska ...,https://www.huffingtonpost.com/entry/kara-east...,Kara Eastman prevailed against an establishmen...,kara-eastman-wins-nebraska-democratic-primary,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
44905,SPORTS,This Baseball Team Learned There's AÂ Wrong Wa...,https://www.huffingtonpost.com/entry/san-jose-...,Many fans were pissed after seeing the minor l...,san-jose-giants-japanese-heritage-night,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44906,SPORTS,Some Young Spurs Fan Dabbed 38 Times In A Sing...,https://www.huffingtonpost.com/entry/dab-kid-s...,"Never change, young man. Never change.",dab-kid-san-antonio-spurs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44907,SPORTS,Rasheed Wallace Ejected From Knicks-Suns Game ...,https://www.huffingtonpost.com/entry/rasheed-w...,Wallace was hit with a first technical for a h...,rasheed-wallace-ejected-knicks-suns-ball-dont-lie,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44908,SPORTS,Why Jake Plummer And Other NFL Players Are Pus...,https://www.huffingtonpost.comhttp://extras.de...,They believe CBD could be an alternative to po...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
News_DF = News_DF.drop(columns=['Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11'])

In [177]:
News_DF

,category,headline,links,short_description,keywords
0,POLITICS,GOP Congressman Suggests Looser Standard Of Pr...,https://www.huffingtonpost.com/entry/zeldin-mu...,"Rep. Lee Zeldin says there needs to be ""some t...",zeldin-muslims-reasonable-suspicion
1,POLITICS,The ACLU Is Suing To Strike Down Ohio's Congre...,https://www.huffingtonpost.com/entry/ohio-cong...,The civil liberties group says Ohio's map is s...,ohio-congressional-map-gerrymandering
2,POLITICS,Don't Like The New Climate Rules? Thank Congress.,https://www.huffingtonpost.com/entry/congress-...,Legislators could have passed a climate change...,congress-climate-rules-emissions
3,POLITICS,The Story Behind An Iconic Picture Of Civil Ri...,https://www.huffingtonpost.com/entry/how-julia...,An often-forgotten battle for free speech righ...,how-julian-bond-fought-georgia-all-the-way-to-...
4,POLITICS,Pro-Abortion Rights Progressive Wins Nebraska ...,https://www.huffingtonpost.com/entry/kara-east...,Kara Eastman prevailed against an establishmen...,kara-eastman-wins-nebraska-democratic-primary
...,...,...,...,...,...
44905,SPORTS,This Baseball Team Learned There's AÂ Wrong Wa...,https://www.huffingtonpost.com/entry/san-jose-...,Many fans were pissed after seeing the minor l...,san-jose-giants-japanese-heritage-night
44906,SPORTS,Some Young Spurs Fan Dabbed 38 Times In A Sing...,https://www.huffingtonpost.com/entry/dab-kid-s...,"Never change, young man. Never change.",dab-kid-san-antonio-spurs
44907,SPORTS,Rasheed Wallace Ejected From Knicks-Suns Game ...,https://www.huffingtonpost.com/entry/rasheed-w...,Wallace was hit with a first technical for a h...,rasheed-wallace-ejected-knicks-suns-ball-dont-lie
44908,SPORTS,Why Jake Plummer And Other NFL Players Are Pus...,https://www.huffingtonpost.comhttp://extras.de...,They believe CBD could be an alternative to po...,NaN


In [178]:
small_data = News_DF.sample(n=40000, random_state=1,ignore_index=True)
df_train =  small_data[:32000]
df_test = small_data[32001:]
df_test = df_test.reset_index(drop=True)

check for the balanced category of train and test dataset:

In [179]:
df_train.groupby(['category']).count()

,headline,links,short_description,keywords
category,,,,
BUSINESS,3533,3533,3533,3186
ENTERTAINMENT,3545,3545,3545,3402
FOOD & DRINK,3546,3546,3546,3448
PARENTING,3568,3568,3568,3195
POLITICS,3575,3575,3574,3341
SPORTS,3585,3585,3585,3403
STYLE & BEAUTY,3541,3541,3541,3335
TRAVEL,3566,3566,3566,3462
WORLD NEWS,3541,3541,3541,3423


In [180]:
df_test.groupby(['category']).count()

,headline,links,short_description,keywords
category,,,,
BUSINESS,905,905,905,820
ENTERTAINMENT,867,867,867,829
FOOD & DRINK,909,909,909,889
PARENTING,874,874,874,770
POLITICS,864,864,864,818
SPORTS,857,857,857,819
STYLE & BEAUTY,924,924,924,876
TRAVEL,907,907,907,884
WORLD NEWS,892,892,892,860


In [181]:
import re
from spacy.language import Language
import spacy

pipeline = spacy.load('en_core_web_sm')


# http://emailregex.com/
email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

# replace = [ (pattern-to-replace, replacement),  ...]
replace = [
    (r"<a[^>]*>(.*?)</a>", r"\1"),  # Matches most URLs
    (email_re, "email"),            # Matches emails
    (r"(?<=\d),(?=\d)", ""),        # Remove commas in numbers
    (r"\d+", "numbr"),              # Map digits to special token <numbr>
    (r"[\t\n\r\*\.\@\,\-\/]", " "),   # Punctuation and other junk
    (r"\s+", " ")                   # Stips extra whitespace
]

train_text = df_train['headline']
test_text = df_test['headline']
for repl in replace:
    train_text = [re.sub(repl[0], repl[1], text) for text in train_text]
    test_text = [re.sub(repl[0], repl[1], text) for text in test_text]

@Language.component("db14")
def ng20_preprocess(doc):
    tokens = [token for token in doc 
              if not any((token.is_stop, token.is_punct))]
    tokens = [token.lemma_.lower().strip() for token in tokens]
    tokens = [token for token in tokens if token]
    return " ".join(tokens)

pipeline.add_pipe("db14")

<function __main__.ng20_preprocess(doc)>

In [182]:
docs_train = [pipeline(doc) for doc in train_text]
docs_test = [pipeline(doc) for doc in test_text]

In [72]:
docs_train[0]

'falkland island war veterans continue fight buenos aires'

In [24]:
df_train['headline'][0]

'Falkland Island War Veterans Continue The Fight In Buenos Aires'

# Part 2: Multinomial Naive Bayes

In [185]:
params = {'alpha': [0.5, 0.7,1,2],
         }
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf',GridSearchCV(MultinomialNB(), param_grid=params, n_jobs=-1, cv=5, verbose=5)),
])
# Fitting our train data to the pipeline
text_clf.fit(docs_train, df_train['category'])


# Predicting our test data
predicted = text_clf.predict(docs_test)
print(classification_report(df_test['category'],predicted ))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
                precision    recall  f1-score   support

      BUSINESS       0.75      0.76      0.76       905
 ENTERTAINMENT       0.79      0.74      0.77       867
  FOOD & DRINK       0.88      0.86      0.87       909
     PARENTING       0.74      0.76      0.75       874
      POLITICS       0.75      0.74      0.75       864
        SPORTS       0.83      0.91      0.87       857
STYLE & BEAUTY       0.85      0.84      0.84       924
        TRAVEL       0.81      0.78      0.80       907
    WORLD NEWS       0.79      0.80      0.80       892

      accuracy                           0.80      7999
     macro avg       0.80      0.80      0.80      7999
  weighted avg       0.80      0.80      0.80      7999



In [166]:
print(text_clf[2].best_params_)

{'alpha': 0.5}


# Part 3: Perceptron

In [184]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', Perceptron(tol=1e-3, random_state=0)),
])
# Fitting our train data to the pipeline
text_clf.fit(docs_train, df_train['category'])

# Predicting our test data
predicted = text_clf.predict(docs_test)
print(classification_report(df_test['category'],predicted ))

                precision    recall  f1-score   support

      BUSINESS       0.75      0.80      0.77       905
 ENTERTAINMENT       0.78      0.66      0.72       867
  FOOD & DRINK       0.80      0.83      0.81       909
     PARENTING       0.70      0.74      0.72       874
      POLITICS       0.71      0.69      0.70       864
        SPORTS       0.85      0.91      0.88       857
STYLE & BEAUTY       0.82      0.81      0.82       924
        TRAVEL       0.74      0.75      0.75       907
    WORLD NEWS       0.76      0.72      0.74       892

      accuracy                           0.77      7999
     macro avg       0.77      0.77      0.77      7999
  weighted avg       0.77      0.77      0.77      7999



# Part 4: SVM

In [183]:
param_grid = {'C': np.arange(0.1,1,0.1)}
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', GridSearchCV(LinearSVC(tol=1e-5),param_grid,refit=True,verbose=2)),
])
# Fitting our train data to the pipeline
text_clf.fit(docs_train, df_train['category'])

# Predicting our test data
predicted = text_clf.predict(docs_test)
print(classification_report(df_test['category'],predicted ))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ..............................................C=0.1; total time=   0.2s
[CV] END ..............................................C=0.1; total time=   0.2s
[CV] END ..............................................C=0.1; total time=   0.3s
[CV] END ..............................................C=0.1; total time=   0.2s
[CV] END ..............................................C=0.1; total time=   0.2s
[CV] END ..............................................C=0.2; total time=   0.2s
[CV] END ..............................................C=0.2; total time=   0.2s
[CV] END ..............................................C=0.2; total time=   0.2s
[CV] END ..............................................C=0.2; total time=   0.2s
[CV] END ..............................................C=0.2; total time=   0.2s
[CV] END ..............................C=0.30000000000000004; total time=   0.3s
[CV] END ..............................C=0.300000

In [169]:
print(text_clf[2].best_estimator_)

LinearSVC(C=0.5)


# Part 5: Decision Tree

In [186]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', DecisionTreeClassifier(criterion="gini", 
                             splitter='random'))
])
# Fitting our train data to the pipeline
text_clf.fit(docs_train, df_train['category'])

# Predicting our test data
predicted = text_clf.predict(docs_test)

print(classification_report(df_test['category'],predicted ))

                precision    recall  f1-score   support

      BUSINESS       0.74      0.78      0.76       905
 ENTERTAINMENT       0.63      0.57      0.60       867
  FOOD & DRINK       0.73      0.69      0.71       909
     PARENTING       0.57      0.71      0.63       874
      POLITICS       0.66      0.56      0.61       864
        SPORTS       0.81      0.85      0.83       857
STYLE & BEAUTY       0.74      0.70      0.72       924
        TRAVEL       0.58      0.63      0.60       907
    WORLD NEWS       0.68      0.63      0.65       892

      accuracy                           0.68      7999
     macro avg       0.68      0.68      0.68      7999
  weighted avg       0.68      0.68      0.68      7999



# Part 6: Random Forest

In [172]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier(criterion ='gini',random_state=1,n_estimators=100))
])
# Fitting our train data to the pipeline
text_clf.fit(docs_train, df_train['category'])

# Predicting our test data
predicted = text_clf.predict(docs_test)

print(classification_report(df_test['category'],predicted ))

                precision    recall  f1-score   support

      BUSINESS       0.77      0.72      0.75       828
 ENTERTAINMENT       0.73      0.61      0.66       816
  FOOD & DRINK       0.78      0.74      0.76       814
     PARENTING       0.57      0.73      0.64       785
      POLITICS       0.73      0.62      0.67       790
        SPORTS       0.85      0.87      0.86       790
STYLE & BEAUTY       0.77      0.74      0.76       787
        TRAVEL       0.68      0.67      0.68       804
      WELLNESS       0.51      0.65      0.57       782
    WORLD NEWS       0.75      0.69      0.72       803

      accuracy                           0.70      7999
     macro avg       0.71      0.70      0.71      7999
  weighted avg       0.72      0.70      0.71      7999



# Part 7: LSTM

In [111]:
max_words = 8192
phrase_len =df_train['headline'].apply(lambda p: len(p.split(' ')))
max_phrase_len = phrase_len.max()
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D
model_lstm = Sequential()
model_lstm.add(Embedding(input_dim = max_words, output_dim = 256, input_length = max_phrase_len))
model_lstm.add(SpatialDropout1D(0.3))
model_lstm.add(LSTM(256, dropout = 0.3, recurrent_dropout = 0.3))
model_lstm.add(Dense(256, activation = 'relu'))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(5, activation = 'softmax'))

In [126]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
tokenizer = Tokenizer(
    num_words = max_words,
    filters = '"#$%&()*+-/:;<=>@[\]^_`{|}~'
)
tokenizer.fit_on_texts(docs_train)

X_train = tokenizer.texts_to_sequences(docs_train)
X_train = pad_sequences(X_train, maxlen = max_phrase_len)
y_train = to_categorical(df_train['category'])

history = model_lstm.fit(
    x_train,
    y_train,
    validation_split = 0.1,
    epochs = 8,
    batch_size = 512
)

ImportError: cannot import name 'to_categorical' from 'keras.utils' (D:\anaconda3\envs\tensorflow1\lib\site-packages\keras\utils\__init__.py)